In [ ]:
import os
import sys
import librosa
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import numpy as np
import mysql.connector as mysql
import bokeh.plotting
import bokeh.models
import bokeh.io
import datetime
import pandas as pd
import math
import re
import statistics
from scipy.signal import find_peaks

In [ ]:
datetimeTickFormatter = bokeh.models.DatetimeTickFormatter(
    microseconds = ['%d-%m    %H:%M:%S.%3N'],
    milliseconds = ['%d-%m    %H:%M:%S.%3N'],
    seconds = ['%d-%m    %H:%M:%S.%3N'],
    minsec = ['%d-%m    %H:%M:%S.%3N'],
    minutes = ['%d-%m    %H:%M:%S.%3N'],
    hourmin = ['%d-%m    %H:%M:%S.%3N'],
    hours = ['%d-%m    %H:%M:%S.%3N'],
    days = ['%d-%m    %H:%M:%S.%3N'],
    months = ['%d-%m    %H:%M:%S.%3N'],
    years = ['%d-%m    %H:%M:%S.%3N'])

def getCombinedWavs(directory, use_directory_starttime=True):
    nr = 0
    sr = None
    y = None
    files = os.listdir(directory)
    files.sort()
    last_endtime = None
    total_gap = 0
    for filename in files:
        fullfilename = f'{directory}/{filename}'
        y_part, sr_part = librosa.load(fullfilename, sr=None)
        
        file_end = int(re.search("^\d*-(\d*)\.wav", filename).group(1))
        file_start = file_end - int(len(y_part) * 1000 / sr_part)
        if sr == None:
            sr = sr_part
            y = y_part
            end = int(re.search("^\d*-(\d*)\.wav", files[0]).group(1))
            if use_directory_starttime:
                unixtime_start = int(re.search(".*-(\d*)", directory).group(1))
            else:
                unixtime_start = file_start            
        else:
            if sr != sr_part:
                raise Exception("Sampling rate mismatch")
            y = np.concatenate([y, y_part])
        nr += 1
        if last_endtime == None:
            print (f'part {"%3d" % nr}, {"%7d" % len(y_part)} samples from unixtime {file_start} to {file_end}.')
        else:
            gap = file_start - last_endtime
            total_gap += gap
            print (f'part {"%3d" % nr}, {"%7d" % len(y_part)} samples from unixtime {file_start} to {file_end}. Gap: {gap}. Total gap: {total_gap}')
        last_endtime = file_end
        
    unixtime_end = int(unixtime_start + len(y) * 1000 / sr)
    print (f'total:    {"%7d" % len(y)} samples from unixtime {unixtime_start} to {unixtime_end}')
    print (f'Read {nr} parts from {directory}. {y.shape[0]} samples: {"%.1f" % (y.shape[0]/sr)} seconds at {sr} samples/sec, starting at unixtime {unixtime_start}')
    return y, sr, unixtime_start

def queryDb(query, args=()):
    try:
        conn = mysql.connect(user="root",
                             password="1234",
                             host="localhost",
                             port=3306,
                             database="imagedescription")
    except mysql.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)

    # Get Cursor
    cur = conn.cursor()

    cur.execute(query, args)
    data = [x for x in cur]
    cur.close()
    return data

def getSteps(s):
    s = list(s)
    return [cur-prev for (cur, prev) in zip(s, [s[0]]+s)]

def getData(directory, use_directory_starttime=True):
    wav, sr, audio_unixtime_ms = getCombinedWavs(directory, use_directory_starttime)
    audio_length_ms = int(len(wav) / sr * 1000)

    print (f'Recording starts at unix time {audio_unixtime_ms} and lasts until {audio_unixtime_ms + audio_length_ms}')

    events = queryDb("SELECT time, unix_time, type, data "
                     "FROM logger_traces "
                     "WHERE type IN ('keydown', 'keyup') "
                     "AND user_id = %s "
                     "AND unix_time BETWEEN %s AND %s",
                    (8,
                        audio_unixtime_ms,
                        audio_unixtime_ms + audio_length_ms))

    up_events = [x for x in events if x[2]=='keyup']
    down_events = [x for x in events if x[2]=='keydown']

    print(f'Read {len(up_events)} keyup events and {len(down_events)} keydown events.')

    return {
        'wav': wav,
        'sr': sr,
        'audio_unixtime_ms': audio_unixtime_ms,
        'up_events': up_events,
        'down_events': down_events
    }

def plotMatplotlib(data):
    wav=data['wav']
    sr=data['sr']
    audio_unixtime_ms=data['audio_unixtime_ms']
    up_events=data['up_events']
    down_events=data['down_events']
    
    plt.figure(figsize=(12, 4))
    plt.ylim((0,0.02))
    librosa.display.waveplot(wav, sr=sr, alpha=0.1)
    plt.scatter([(x[1]-audio_unixtime_ms)/1000 for x in down_events], [0 for _ in down_events])
    plt.scatter([(x[1]-audio_unixtime_ms)/1000 for x in up_events], [0.01 for _ in up_events])

def plotBokeh(data, adjust_ms=0):
    wav=data['wav']
    sr=data['sr']
    audio_unixtime_ms=data['audio_unixtime_ms']
    up_events=data['up_events']
    down_events=data['down_events']
    
    s_sound = pd.Series(data=wav, index=range(len(wav)))
    df_sound = pd.DataFrame(s_sound)
    df_sound.reset_index(inplace=True)
    df_sound.columns = ['Index', 'wav']
    df_sound['unixtime'] = df_sound['Index'].apply(lambda x: audio_unixtime_ms + x * 1000 / sr)
    df_sound['time'] = df_sound['Index'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x / 1000)))

    bokeh.io.output_notebook()
    p = bokeh.plotting.figure()
    p.xaxis.formatter = datetimeTickFormatter
    p.xaxis.major_label_orientation = math.pi/2
    p.line(x='unixtime', y='wav', source=df_sound)
    
    def plotEvents(events, colour="black", y_offset=0):
        df = pd.DataFrame()
        df['x'] = [e[1]+adjust_ms for e  in events]
        df['y'] = [0.015]*len(events)
        df['text'] = [e[3][3:] if e[3].startswith('Key') else e[3] for e in events]
        source = bokeh.models.ColumnDataSource(df)

        p.scatter(x='x', y='y', source=source, size=10, color=colour, alpha=0.5)
        p.add_layout(bokeh.models.LabelSet(x='x', y='y', text='text', source=source, x_offset=5, y_offset=y_offset, render_mode='canvas', angle=math.pi/2, text_color=colour))
    
    plotEvents(down_events, "green", 20)
    plotEvents(up_events, "red", -120)
       
    bokeh.plotting.show(p)
    
def diffPeekAmplAndKeyDown(directory):
    data = getData(directory)
    down_event = data['down_events'][0][1]
    max_index = np.argmax(data['wav'])
    max_time = data['audio_unixtime_ms'] + int(max_index * 1000 / data['sr'])
    print (f'{directory} max ampl at {max_time}, keydown at {down_event}, difference {max_time-down_event}')

def getDataSubset(data, offset_ms, length_ms=2000):
    sr = data['sr']
    starttime = data['audio_unixtime_ms'] + offset_ms
    subset_wav = data['wav'][int(offset_ms*sr/1000):int((offset_ms+length_ms)*sr/1000)]
    subset_up_events = [e for e in data['up_events'] if starttime < e[1] and e[1] < (starttime+length_ms)]
    subset_down_events = [e for e in data['down_events'] if starttime < e[1] and e[1] < (starttime+length_ms)]
    return {
        'wav': subset_wav,
        'sr': sr,
        'audio_unixtime_ms': starttime,
        'up_events': subset_up_events,
        'down_events': subset_down_events
    }
    
def aap(x, adjust_ms=0):
    data=getData(f'../web/data/videos/{x}')
    plotBokeh(data, adjust_ms=adjust_ms)
    
    wav=data['wav']
    sr=data['sr']
    s=librosa.stft(y=wav, hop_length=int(sr/1000))
    data['wav'] = abs(s.sum(axis=0))
    data['sr'] = 1000
    plotBokeh(data)

def moving_ranges(data, context):
    data2 = list(data)
    data3 = [data2[0]]*context + data2 + [data2[-1]]*context
    return [data3[i:i+1+2*context] for i in range(len(data2))]
def moving_ranges_f(data, context, f):
    ranges = moving_ranges(data, context)
    return [f(x) for x in ranges]
def moving_average(x, context):
    return moving_ranges_f(x, context, statistics.mean)
def moving_var(x, context):
    return moving_ranges_f(x, context, lambda r: max(r)-min(r))

def getAllTraceEvents(directory):
    starttime=int(re.search(".*-(\d*)", directory).group(1))
    return queryDb(f"""select * from logger_traces where uuid = (select uuid from logger_traces lt where type='startaudio' and data like '%{starttime}') order by id""", ())

def getLostAudioSamplesData(directory):
    events = getAllTraceEvents(directory)
    timeAndBufferSize = [(x[6], x[8]) for x in events if x[7] == 'recorder-mark'][1:]
    starttime=timeAndBufferSize[0][0]
    startsize=timeAndBufferSize[0][1]
    times = [t for (t, s) in timeAndBufferSize]
    samplesDiff = [(s-startsize)-((t-starttime)*48000/1000) for (t, s) in timeAndBufferSize]
    msSinceLastSample=getSteps(times)
    df = pd.DataFrame()
    df['time']=times
    df['samplesDiff']=samplesDiff
    df['msSinceLastSample']=msSinceLastSample
    return df
    
# def plotLostAudioSamples(directory):
#     (times, samplesDiff, msSinceLastSample) = getLostAudioSamplesData(directory)
#     bokeh.io.output_notebook()
#     p = bokeh.plotting.figure()
    
#     p.xaxis.formatter = datetimeTickFormatter
#     p.xaxis.major_label_orientation = math.pi/2
#     p.line(x=times, y=samplesDiff, color="blue")
#     p.line(x=times, y=msSinceLastSample, color="red")
#     bokeh.plotting.show(p)
def plotLostAudioSamples(directory):
    df = getLostAudioSamplesData(directory)
    bokeh.io.output_notebook()
    p = bokeh.plotting.figure()
    
    p.xaxis.formatter = datetimeTickFormatter
    p.xaxis.major_label_orientation = math.pi/2
    p.line(x='time', y='samplesDiff', source=df, color="blue")
    p.line(x='time', y='msSinceLastSample', source=df, color="red")
    bokeh.plotting.show(p)

In [ ]:
# getData('../web/data/videos/niels-step_5-1617191574728')
# plotLostAudioSamples('niels-step_5-1617187925912')
# plotLostAudioSamples('niels-step_5-1617192257518')

In [ ]:
def plotLostAudioSamples3(directory):
    CONTEXT=2
    df = getLostAudioSamplesData(directory)
    
    bokeh.io.output_notebook()
    p = bokeh.plotting.figure()
    p.xaxis.formatter = datetimeTickFormatter
    p.xaxis.major_label_orientation = math.pi/2
#     p.line(x='time', y='samplesDiff', source=df, color="blue")
#     p.line(x='time', y='msSinceLastSample', source=df, color="red")

    # The difference between the number of samples in the data, and the expected number based on sampling rate and during
    # varies over time. Samples don't get delivered exactly on time, but usually the difference stays within a clear bound.
    #
    # Two things may occur that break this pattern:
    #  1) occassionally some samples get queued up and delivered late, so we see a temporary increase in the difference which quickly gets corrected when the samples do arrive
    #  2) on other occassions samples really get dropped and we see the difference increase permanently
    #
    # To keep keystrokes in sync, the first case doesn't matter since later we will only use the samples and don't care when they were received,
    # but for the second case we want to insert some dummy samples to make sure the samples after that stay in sync.
    
    # First find the peaks and dips in the oscilating sample difference
    peaks = find_peaks(df['samplesDiff'])[0]
    dips = find_peaks([-x for x in df['samplesDiff']])[0]
    dips_and_peaks = np.concatenate((dips, peaks))
    dips_and_peaks.sort()
    df_dips_peaks = df.iloc[dips_and_peaks].copy()
    df_dips_peaks['dippeak'] = ['dip' if x in dips else 'peak' for x in df_dips_peaks.index]
    
    # Plot these points, which will contain some outlier for case 1) above
    p.scatter(x='time', y='samplesDiff', source=df_dips_peaks, color="yellow")
    df_dips_peaks['samplesDiffMV'] = moving_var(df_dips_peaks['samplesDiff'], CONTEXT)
    p.line(x='time', y='samplesDiffMV', source=df_dips_peaks, color="yellow")
    
    # Filter out these outliers based on the max difference in the samplesDiff column over a window of +/- CONTEXT samples
    median_moving_var = statistics.median(moving_var(df_dips_peaks['samplesDiff'], CONTEXT))
    print(f'median moving var: {median_moving_var}')
    not_outliers = [i for i in range(len(dips_and_peaks)-1)
                    if df_dips_peaks['samplesDiff'].iloc[i+1] - df_dips_peaks['samplesDiff'].iloc[i] < 1.5*median_moving_var]
    df_dips_peaks_clean = df_dips_peaks.iloc[not_outliers].copy()

    # Then plot the dips and peaks again for the cleaned data
    p.scatter(x='time', y='samplesDiff', source=df_dips_peaks_clean, color="blue")
    # recalculate this since it will have changed after removing the outliers
    df_dips_peaks_clean['samplesDiffMV'] = moving_var(df_dips_peaks_clean['samplesDiff'], CONTEXT)     
    p.line(x='time', y='samplesDiffMV', source=df_dips_peaks_clean, color="blue")
 
    # We now want to find the 'blocks' of stable data, and calculate the number samples lost by comparing the average value for peaks and dips in two blocks
    # The variation in sample difference is quite sample within a block, so we mark points where this variation exceed 110% of the median variance over a +/- CONTEXT window
    samples_lost_at = [i for i in range(1, len(df_dips_peaks_clean))
                if ((df_dips_peaks_clean['samplesDiffMV'].iloc[i-1] < median_moving_var*1.1)
                and (df_dips_peaks_clean['samplesDiffMV'].iloc[i]   >= median_moving_var*1.1))]
    df_samples_lost_at = df_dips_peaks_clean.iloc[samples_lost_at].copy()
    # Plot the points where data is lost in red
    p.scatter(x='time', y='samplesDiffMV', source=df_samples_lost_at, color="red")
    
    # Determine a list of intervals where the dips and peaks are stable based on the points where samples are lost,
    # adding the first and last samples as endpoints
    intervals = pd.DataFrame(zip([min(df_dips_peaks_clean['time'])] + list(df_samples_lost_at['time']),
                                 list(df_samples_lost_at['time'])+[max(df_dips_peaks_clean['time'])]))
    intervals.columns = ['from', 'to']
    # Calculate the Mean value for dips and peaks over each interval
    def getMeanDipsPeaksForInterval(row, dippeak):
        samples =  df_dips_peaks_clean[df_dips_peaks_clean.time.between(row['from'], row['to']) 
                                        & (df_dips_peaks_clean.samplesDiffMV < median_moving_var*1.1)
                                        & (df_dips_peaks_clean.dippeak==dippeak)]['samplesDiff']
        return statistics.mean(samples) if len(samples) > 0 else np.NaN
    intervals['avg_peak'] = intervals.apply(lambda row: getMeanDipsPeaksForInterval(row, 'peak'), axis=1)
    intervals['avg_dip'] = intervals.apply(lambda row: getMeanDipsPeaksForInterval(row, 'dip'), axis=1)
    # Calculate the required adjustment based on the difference in mean peaks and dips for all but the first interval
    intervals.insert(0, 'adjustment', 0)
    for i in range (1,len(intervals)):
        diff = [intervals['avg_peak'].iloc[i-1] - intervals['avg_peak'].iloc[i],
                intervals['avg_dip'].iloc[i-1] - intervals['avg_dip'].iloc[i]]
        diff = [i for i in diff if not math.isnan(i)]
        if len(diff) > 0:
            intervals['adjustment'].iloc[i] = statistics.mean(diff)
    
    # Create new dataframe for visualisation only to show the dips and peaks after adjustment
    df_adjusted = df_dips_peaks_clean.copy()
    df_adjusted['adjusted'] = df_dips_peaks_clean['samplesDiff'].copy()
    for i in range(1, len(intervals)):
        from_time = intervals['from'].iloc[i]
        adjustment = intervals['adjustment'].iloc[i]
        for index, _ in df_adjusted.iterrows():
            if from_time <= df_adjusted.loc[index]['time']:
                if index==489:
                    print(f"adjust {index} {df_adjusted.loc[index]['samplesDiff']} +{adjustment}")
                df_adjusted.at[index, 'adjusted'] += adjustment
    p.scatter(x='time', y='adjusted', source=df_adjusted, color="green")
    
    bokeh.plotting.show(p)
    return df_adjusted, df_samples_lost_at, intervals

df_adjusted, df_samples_lost_at, intervals = plotLostAudioSamples3('niels-step_5-1617192257518')

In [ ]:
plotLostAudioSamples3('niels-step_5-1617191574728')
plotLostAudioSamples3('niels-step_5-1617187925912')
plotLostAudioSamples3('niels-step_5-1617192257518')